In [ ]:
!pip install google-genai

In [ ]:
import os
import time
import json
from pydantic import BaseModel, Field
from google import genai
from google.genai import types

# ---------------- 配置部分 ----------------

# 请替换为你的 API Key，或者设置环境变量 GEMINI_API_KEY
# https://aistudio.google.com/api-keys
API_KEY = "你自己的API Key"

# 设置目标题目数量
TOTAL_QUESTIONS = 1000
# 每次请求获取的题目数 (建议 20-50 之间，避免超出单次输出 Token 限制)
BATCH_SIZE = 50 
# 输出文件名
OUTPUT_FILE = "math_problems_1000.json"

In [ ]:

# ---------------- 1. 定义极其简单的“模具” ----------------
# 告诉 Gemini：我只要这两个字段，别的都不要
class SimpleQuestion(BaseModel):
    grade: str = Field(description="年级，如 '4年级'")
    content: str = Field(description="数学题目的具体内容，不包含答案")

# ---------------- 主程序 ----------------
def main():
    client = genai.Client(api_key=API_KEY)
    
    all_data = [] # 用来存所有的题目
    
    print(f"开始获取 {TOTAL_QUESTIONS} 道纯数学题 (无答案)...")

    while len(all_data) < TOTAL_QUESTIONS:
        # 计算还要多少道
        needed = TOTAL_QUESTIONS - len(all_data)
        current_batch_size = min(BATCH_SIZE, needed)
        
        print(f"正在请求... (当前进度: {len(all_data)}/{TOTAL_QUESTIONS})")

        # 提示词：明确告诉它不要答案
        prompt = (
            f"请给我 {current_batch_size} 道小学 4-6 年级的数学题。"
            "要求：\n"
            "1. 只要题目，绝对不要答案，也不要选项。\n"
            "2. 题目类型包含计算、应用题、几何。\n"
            "3. 题目描述要清晰。"
        )

        try:
            response = client.models.generate_content(
                model="gemini-2.0-flash", # 建议用 Flash，速度快且便宜，足够生成题目
                contents=prompt,
                config=types.GenerateContentConfig(
                    response_mime_type="application/json",
                    response_schema=list[SimpleQuestion], # 关键：套用上面的简单模具
                    temperature=0.8,
                ),
            )

            if response.parsed:
                # 把这一批题目加到总列表里
                for item in response.parsed:
                    all_data.append(item.model_dump())
            
        except Exception as e:
            print(f"出错重试: {e}")
            time.sleep(2)
            continue
            
        time.sleep(1) # 稍微歇一下，防止请求太快

    # ---------------- 保存文件 ----------------
    # 构造成你想要的格式 { "questions": [ ... ] }
    final_json = {
        "questions": all_data
    }

    with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
        json.dump(final_json, f, ensure_ascii=False, indent=2)

    print(f"完成！已保存到 {OUTPUT_FILE}")

if __name__ == "__main__":
    main()

In [ ]:
INPUT_FILE = "math_dataset_with_answers.jsonl"
OUTPUT_FILE = "incorrect_analysis.json"

# 使用逻辑能力最强的 Pro 版本
MODEL_NAME = "gemini-2.5-pro" 

# ---------------- 1. 定义新的输出结构 ----------------

# 定义单个错误项：包含 ID 和 原因
class IncorrectItem(BaseModel):
    id: int = Field(description="题目对应的 ID 编号")
    reason: str = Field(description="简短解释为什么这道题是错的（例如：'计算错误，3+5应为8' 或 '逻辑推导有误'）")

# 定义最终返回的列表结构
class ReviewResult(BaseModel):
    incorrect_items: list[IncorrectItem]

# ---------------- 主程序 ----------------
def main():
    client = genai.Client(api_key=API_KEY)

    # 1. 读取并组装数据
    if not os.path.exists(INPUT_FILE):
        print(f"找不到文件: {INPUT_FILE}")
        return

    print(f"正在读取 {INPUT_FILE} ...")
    
    full_content_for_prompt = ""
    total_count = 0

    with open(INPUT_FILE, "r", encoding="utf-8") as f:
        for index, line in enumerate(f):
            line = line.strip()
            if not line: continue
            
            try:
                item = json.loads(line)
                # 组装文本，明确标出 ID
                record_str = (
                    f"ID: {index}\n"
                    f"Question: {item.get('instruction')}\n"
                    f"Thinking: {item.get('thinking')}\n"
                    f"Answer: {item.get('output')}\n"
                    f"-----------------------------\n"
                )
                full_content_for_prompt += record_str
                total_count += 1
            except:
                continue

    print(f"组装完成，共 {total_count} 条数据。")
    print(f"正在发送给 {MODEL_NAME} 进行深度检查...")
    print("这可能需要 1-2 分钟，请耐心等待...")

    # 2. 构造提示词
    prompt = (
        "你是一位资深数学老师。请仔细检查以下所有数学题的【推理过程】和【最终答案】。\n"
        "如果发现任何错误（计算错误、逻辑谬误、或者答案错误），请将其记录下来。\n"
        "\n"
        "任务要求：\n"
        "1. 如果题目完全正确，忽略它。\n"
        "2. 如果有错，请记录其 ID，并用简短的中文说明错误原因。\n"
        "3. 严格按照定义的 JSON 格式返回结果。\n"
        "\n"
        "以下是待检查的题目列表：\n"
        f"{full_content_for_prompt}"
    )

    try:
        # 3. 调用 API
        response = client.models.generate_content(
            model=MODEL_NAME,
            contents=prompt,
            config=types.GenerateContentConfig(
                response_mime_type="application/json",
                response_schema=ReviewResult, # 关键：使用新的结构
                temperature=0.1, # 低温度保证严谨
            ),
        )

        # 4. 解析结果
        if response.parsed:
            errors = response.parsed.incorrect_items
            
            # 按 ID 排序，方便查看
            errors.sort(key=lambda x: x.id)
            
            print("-" * 50)
            if not errors:
                print("🎉 太棒了！Gemini 没有发现任何错误，全对！")
            else:
                print(f"检查完成！共发现 {len(errors)} 个错误：\n")
                
                # 打印到控制台预览
                for item in errors[:10]: # 只打印前10个避免刷屏
                    print(f"[ID: {item.id}] ❌ {item.reason}")
                
                if len(errors) > 10:
                    print(f"... 还有 {len(errors)-10} 个错误未显示。")

                # 转换为字典列表以便保存
                save_data = [item.model_dump() for item in errors]

                # 保存到文件
                with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
                    json.dump(save_data, f, ensure_ascii=False, indent=2)
                
                print(f"\n完整错误报告已保存至: {OUTPUT_FILE}")
            
        else:
            print("API 返回内容无法解析。")
            print("原始返回:", response.text)

    except Exception as e:
        print(f"❌ 请求发生错误: {e}")

if __name__ == "__main__":
    main()

In [ ]:
client = genai.Client(api_key=API_KEY)

print("正在列出可用模型...")
try:
    # 新版 SDK 的 list 方法返回的是一个迭代器
    for model in client.models.list():
        # 直接打印 model.name，不进行属性过滤，防止报错
        print(f"- {model.name}")
        # 如果你想看它的所有属性，可以用 print(model.model_dump())
except Exception as e:
    print(f"查询出错: {e}")